In [1]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import shutil
import sys

import flopy
from modflowapi import ModflowApi
from modflowapi.extensions import ApiSimulation

from bmi.wrapper import BMIWrapper

import pyswmm
from pyswmm import Simulation, Nodes
from pyswmm import Output

C:\Users\jdhughes\miniconda3\envs\liss\lib\site-packages\bmi\__init__.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
sys.path.append("../common")
from liss_settings import \
    libmf6, \
    get_dflow_grid_name, get_dflow_dtuser, \
    get_modflow_coupling_tag, get_modflow_grid_name, \
    silent, verbosity, \
    print_path, print_value

In [3]:
control_path = pl.Path("../dflow-fm/coarse/tides/base/FlowFM.mdu") # change this if using a different D-Flow FM control file
grid_name = get_dflow_grid_name(control_path)
print(grid_name)

LIS_modflow_bathy2_net


In [4]:
swmm_path = pl.Path("../swmm/greenport/greenport_detailedsewer_v4_nosub175.inp").resolve()
print(swmm_path, swmm_path.is_file())

\\Mac\Documents\Work\compound_flooding\swmm\greenport\greenport_detailedsewer_v4_nosub175.inp True


In [5]:
fpth = swmm_path.with_suffix(".out")
if fpth.is_file():
    print(f"removing: '{fpth}'")
    fpth.unlink()
fpth = swmm_path.with_suffix(".rpt")
if fpth.is_file():
    print(f"removing: '{fpth}'")
    fpth.unlink()


In [6]:
mf_grid_name = get_modflow_grid_name()
print(mf_grid_name)

greenport_chd


In [7]:
dflowfm_dtuser = get_dflow_dtuser(control_path)
print(dflowfm_dtuser)

300.0


#### Set unit conversion factors

In [8]:
d2sec = 24. * 60. * 60.
hrs2sec = 60. * 60. 
m2ft = 3.28081
cfd2cms = 1.0 / ((m2ft**3) * 86400.)

#### Set the MODFLOW coupling frequency

Change the `mf_couple_freq_hours` value. Only tested for multiple of the D-Flow FM DtUser variable. Will not work for `mf_couple_freq_hours` values greater than 24.

In [9]:
mf_couple_freq_hours = 24.0  #Change this value to change the coupling frequency
mf_couple_freq = mf_couple_freq_hours * hrs2sec
dflow_per_mf = int(mf_couple_freq / dflowfm_dtuser)

In [10]:
print(f"MODFLOW coupling frequency {mf_couple_freq_hours} hours\nMODFLOW coupled to D-FLOW FM every {dflow_per_mf} output time step ({dflowfm_dtuser} sec.)") 

MODFLOW coupling frequency 24.0 hours
MODFLOW coupled to D-FLOW FM every 288 output time step (300.0 sec.)


In [11]:
mf_tag = get_modflow_coupling_tag(mf_couple_freq_hours)
print(f"MODFLOW coupling tag: {mf_tag}")

MODFLOW coupling tag: 01.00D


In [12]:
nstp = int(86400.0 / (dflow_per_mf * dflowfm_dtuser))
print(f"MODFLOW time steps per day: {nstp}")

MODFLOW time steps per day: 1


#### Set a few variables for controlling coupling

In [13]:
HDRY = -1e30
DEPTH_MIN = 0.1

#### Print the path of the modflow6 shared library

In [14]:
str(libmf6), libmf6.is_file()

('C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Scripts\\libmf6.dll', True)

#### Load the D-FLOW to MODFLOW weights

GHB weights

In [15]:
fpath = f"../mapping/dflow_{grid_name}_to_{mf_grid_name}_ghb.npz"
npzfile = np.load(fpath)
npzfile

NpzFile '../mapping/dflow_LIS_modflow_bathy2_net_to_greenport_chd_ghb.npz' with keys: dflow2mfghb, ghbmask, ghb2qext

In [16]:
dflow2mfghb = npzfile["dflow2mfghb"]
ghbmask = npzfile["ghbmask"]
ghb2qext = npzfile["ghb2qext"]

CHD weights

In [17]:
fpath = f"../mapping/dflow_{grid_name}_to_{mf_grid_name}_chd.npz"
npzfile = np.load(fpath)
npzfile

NpzFile '../mapping/dflow_LIS_modflow_bathy2_net_to_greenport_chd_chd.npz' with keys: dflow2mfchd, chdmask, chd2qext

In [18]:
dflow2mfchd = npzfile["dflow2mfchd"]
chdmask = npzfile["chdmask"]
chd2qext = npzfile["chd2qext"]

#### Define paths for the model simulation

In [19]:
mf_base_path = pl.Path("../modflow/greenport_chd/base/").resolve()
mf_run_path = pl.Path(f"../modflow/greenport_chd/run_{mf_tag}/").resolve()

#### Load the base MODFLOW model

In [20]:
sim = flopy.mf6.MFSimulation.load(sim_ws=mf_base_path, verbosity_level=verbosity())
gwf = sim.get_model()

In [21]:
sim.set_sim_path(mf_run_path)

#### Change TDIS to defined time steps

In [22]:
tdis = sim.get_package("TDIS")
perioddata = tdis.perioddata.array

In [23]:
perioddata["nstp"] = nstp
tdis.perioddata = perioddata

#### Write the new model files

In [24]:
sim.write_simulation(silent=silent())

#### Define base GHB variables

In [25]:
ghb_data0 = gwf.ghb.stress_period_data.get_dataframe()[0]
print_value(ghb_data0)

#### Define base CHD variables

In [26]:
chd_surface = gwf.get_package("chd_surface")
chd_data0 = chd_surface.stress_period_data.get_dataframe()[0]
print_value(chd_data0)

#### SWMM data

In [27]:
# SWMM mappings
junctions = (
     "9",
     "10",
     "47",
     "54",
     "44",
     "34",
     "81",
     "155",
     "67",
     "69",
     "30",
     "118",
)

# these are the junction elevations in the SWMM inp file
swmm_inverts = {
    "9": 5,
    "10": 2,
    "47": 2.2,
    "54": 0.7,
    "44": 2.5,
    "34": 6.0,
    "81": 0.7,
    "155": 3,
    "67": -1.0,
    "69": 1.0,
    "30":  5.5,
    "118": 4.6,
 }

# MODFLOW -----------------
# map of the junction names and the MODFLOW (row, column)
# containing the junction
mf6_cells = {
    "9": (25, 32),
    "10": (24, 32),
    "47": (28, 33),
    "54": (29, 33),
    "44": (27, 33),
    "34": (28, 31),
    "81": (28, 30),
    "155": (28, 29),
    "67": (31, 32),
    "69": (31, 31),
    "30": (27, 31),
    "118": (35, 26),
}

#### Setup and initialize D-FLOW FM

You will need to set `dflow_dirpath` to the correct directory on your machine.

In [28]:
# dflow_root_path = pl.Path(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM").resolve()
# dll_paths = (
#     r"plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin",
#     r"plugins\DeltaShell.Dimr\kernels\x64\share\bin",
# )

In [29]:
# # os.environ["PATH"] = (
# #     str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
# # )
# old_path = os.environ["PATH"]
# os.environ["PATH"] = ""
# path = ""
# for p in dll_paths:
#     path += f"{dflow_root_path / p}" + os.pathsep
# os.environ["PATH"] = path + old_path

In [30]:
print_path()

In [31]:
# dflow_dirpath = dflow_root_path / f"{dll_paths[0]}"

In [32]:
# dflow_dirpath = os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin")
# dflow_deps_dirpath = (
#     os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\share\bin"),
# )
dflow_dirpath = pl.Path(r"X:\Work\compound_flooding\dflow-fm\dflowfm_dll") 
dflow_base = pl.Path(r"X:\Work\compound_flooding\dflow-fm\coarse\tides\base").resolve()
dflow_working = pl.Path(r"X:\Work\compound_flooding\dflow-fm\coarse\tides\run").resolve()
dflow_config = dflow_working / "FlowFM.mdu"

In [33]:
if dflow_working.is_dir():
    shutil.rmtree(dflow_working)
shutil.copytree(dflow_base, dflow_working)
(dflow_working / "output").mkdir(parents=True, exist_ok=True)

In [34]:
print_path()

In [35]:
# Add dflowfm dll folder to PATH so that it can be found by the BMIWrapper
os.environ["PATH"] = (
    str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
)

In [36]:
print_path()

In [37]:
(pl.Path(dflow_dirpath) / "dflowfm.dll").is_file()

True

#### Initialize D-Flow FM

In [38]:
dflowfm = BMIWrapper(
    engine="dflowfm",
    configfile=str(dflow_config),
)

In [39]:
dflowfm.initialize()

#### Get data from D-FLOW FM

In [40]:
ndxi = int(dflowfm.get_var("ndxi"))
ndx = int(dflowfm.get_var("ndx"))
x = dflowfm.get_var("xz")
y = dflowfm.get_var("yz")
z = dflowfm.get_var("bl")
xy = [(xx, yy) for (xx, yy) in zip(x, y)]
ndx, ndxi, x.shape, y.shape

(8412, 8323, (8412,), (8412,))

In [41]:
dflowfm.get_var("s1")

array([0., 0., 0., ..., 0., 0., 0.])

In [42]:
v = dflowfm.get_var("hs")
v.shape, v

((8412,),
 array([26.42575318, 24.9297905 , 23.53244183, ..., 18.71800344,
        20.31243107, 21.52212007]))

In [43]:
qext = np.zeros(ndx)
qext.shape, qext

((8412,), array([0., 0., 0., ..., 0., 0., 0.]))

In [44]:
# dflowfm.set_var("qext", qext)

In [45]:
qext_cum = np.zeros(ndx)
qext_cum.shape

(8412,)

In [46]:
vextcum = dflowfm.get_var("vextcum")
vextcum.shape, vextcum

((8412,), array([0., 0., 0., ..., 0., 0., 0.]))

#### Initialize SWMM

In [47]:
swmm_sim = pyswmm.Simulation(str(swmm_path))

# build the swmm_nodes dictionary where the values are the swmm node objects
# for example, "J1 = pyswmm.Nodes(swmm_sim)["J1"]"" , where J1 is the key
# and the value is pyswmm.Nodes(swmm_sim)["J1"]
swmm_nodes = {}
for j in junctions:
    swmm_nodes[j] = pyswmm.Nodes(swmm_sim)[j]

In [48]:
swmm_sim.start()

#### Initialize MODFLOW using MODFLOW API

In [49]:
mf6 = ModflowApi(str(libmf6), working_directory=mf_run_path)

In [50]:
mf6.initialize()

##### use recent modflowapi functionality for accessing data from the api in flopy-like data structures

In [51]:
apisim = ApiSimulation.load(mf6)
apiml = apisim.get_model()

sewer_flow = apiml.get_package("SWMM")
swmm_dtype = [("nodelist", "O"), ("q", float)]

#### Define MODFLOW variable tags and set pointer to MODFLOW variables

In [52]:
ghb_bhead_tag = mf6.get_var_address("BHEAD", "GWF", "GHB")
ghb_cond_tag = mf6.get_var_address("COND", "GWF", "GHB")
ghb_flow_tag = mf6.get_var_address("SIMVALS", "GWF", "GHB")

In [53]:
ghb_bhead_ptr = mf6.get_value_ptr(ghb_bhead_tag)
ghb_cond_ptr = mf6.get_value_ptr(ghb_cond_tag)
ghb_flow = np.zeros(ghb_bhead_ptr.shape)

In [54]:
chd_head_tag = mf6.get_var_address("HEAD", "GWF", "CHD_SURFACE")
chd_flow_tag = mf6.get_var_address("SIMVALS", "GWF", "CHD_SURFACE")

In [55]:
chd_head_ptr = mf6.get_value_ptr(chd_head_tag)
chd_flow = np.zeros(chd_head_ptr.shape)

#### Create dictionaries for saving modified GHB data

In [56]:
ghb_elev_dict = {}
ghb_cond_dict = {}
chd_elev_dict = {}
qext_dict = {}
swmm_q_dict = {}

#### Function to update MODFLOW GHB data

In [57]:
def update_mf(key, s, d):
    mask = d == 0.0
    s[mask] = 0.0
    mult = np.full(d.shape, 1.0)
    mult[mask] = 0.0

    ghb_head = ghb_data0["bhead"].to_numpy()
    ghb_head[ghbmask] = dflow2mfghb.dot(s)[ghbmask] * m2ft
    ghb_cond = ghb_data0["cond"].to_numpy()
    ghb_cond[ghbmask] = ghb_cond[ghbmask] * dflow2mfghb.dot(mult)[ghbmask]
    
    ghb_bhead_ptr[:] = ghb_head[:]
    ghb_cond_ptr[:] = ghb_cond[:]

    chd_head = chd_data0["head"].to_numpy()
    chd_head[chdmask] = dflow2mfchd.dot(s)[chdmask] * m2ft
    
    chd_head_ptr[:] = chd_head[:]
   
    # update results dictionary
    ghb_elev_dict[key] = ghb_head.copy()
    ghb_cond_dict[key] = ghb_cond.copy()
    chd_elev_dict[key] = chd_head.copy()

#### Function to update D-Flow FM Qext data

In [58]:
def update_dflow(key, d):
    ghb_flow = -mf6.get_value(ghb_flow_tag) * cfd2cms
    dflow_qext_ghb = ghb2qext.dot(ghb_flow)
    dflow_qext_ghb[d == 0.0] = 0.0
    
    chd_flow = -mf6.get_value(chd_flow_tag) * cfd2cms
    dflow_qext_chd = chd2qext.dot(chd_flow)
    dflow_qext_chd[d == 0.0] = 0.0

    dflow_qext = dflow_qext_ghb + dflow_qext_chd
    
    qext_cum[:ndxi] += dflow_qext[:ndxi]
    qext[:ndxi] = dflow_qext[:ndxi]
    dflowfm.set_var("qext", qext)

    # update results dictionaries
    qext_dict[key] = qext[:ndxi].copy()


#### Function to update SWMM and the MODFLOW SWMM well

In [59]:
def update_swmm(key):
    heads = apiml.X
    mf6_spd = []
    for key, value in swmm_nodes.items():
        row, col = mf6_cells[key]
        head = heads[0, row, col]
        # might be nice to use the simulated swmm water-level
        # in the junctions instead of the invert elevation
        pot = head - swmm_inverts[key] * m2ft
        if pot > 0.0:
            Q = pot * 0.001
        else:
            Q = pot * 0.0002
        value.generated_inflow(Q * cfd2cms)  # convert to CMS
        mf6_spd.append(((0, row, col), -Q)) # unit conversion needed

    # update gwf_swmm well file with new flux data - will be used next time step
    mf6_spd = np.array(mf6_spd, dtype=swmm_dtype)
    sewer_flow.stress_period_data.values = mf6_spd

    # update results dictionary
    swmm_q_dict[key] = mf6_spd["q"].copy()


#### Run each time step

In [60]:
print(
    f"DFLOWFM current_time: {dflowfm.get_current_time():15,.1f} sec. ({dflowfm.get_current_time()/86400.:15,.1f} days)\n"
     + f"DFLOWFM end_time:     {dflowfm.get_end_time():15,.1f} sec. ({dflowfm.get_end_time()/86400.:15,.1f} days)"
)

DFLOWFM current_time:             0.0 sec. (            0.0 days)
DFLOWFM end_time:         7,776,000.0 sec. (           90.0 days)


In [61]:
idx = 0
jdx = 0
t0 = time.perf_counter()
current_time = dflowfm.get_current_time()
end_time = dflowfm.get_end_time()
while current_time <= end_time:
    idx += 1
    ontime = dflowfm.get_current_time()
    onday = ontime / 86400.
    frac_comp = current_time / end_time
    print(f"Current time: {current_time:15,.1f} ({onday:10.3f} days) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
    dflowfm.update()

    current_time = dflowfm.get_current_time()
    if idx == int(dflow_per_mf):
        print(f"Current time: {current_time:15,.1f} ({onday:10.3f} days) - {frac_comp:6.2%} complete - ({idx:03d}) ***", end="\r")
        s = dflowfm.get_var("s1")[:ndxi]
        d = dflowfm.get_var("hs")[:ndxi]

        mf6.prepare_time_step(mf6.get_time_step())
        update_mf(str(jdx), s, d)
        mf6.do_time_step()
        mf6.finalize_time_step()
        update_dflow(str(jdx), d)
        
        # advance SWMM
        update_swmm(str(jdx))
        dt_sec = mf6.get_time_step() * d2sec
        swmm_sim.step_advance(int(dt_sec))
        try:
            swmm_sim.__next__()  
        except StopIteration:
            break        

        # update counters
        idx = 0
        jdx += 1
    
    if current_time == end_time:
        break

vextcum = dflowfm.get_var("vextcum")

t1 = time.perf_counter()
print(f"\nrun time: {(t1 - t0) / 60.} min")

Current time:     7,776,000.0 (    89.997 days) - 100.00% complete - (288) ***
run time: 56.40523425416662 min


#### Finalize models

In [62]:
mf6.finalize()

In [64]:
swmm_sim.terminate_simulation()
swmm_sim.report()
swmm_sim.close()

In [65]:
dflowfm.finalize()

#### Save ghb elevation and conductance data to compressed files

In [66]:
np.savez_compressed(f"{mf_run_path}/ghb_elev.npz", **ghb_elev_dict)
np.savez_compressed(f"{mf_run_path}/ghb_cond.npz", **ghb_cond_dict)

#### Save chd elevation to compressed file

In [67]:
np.savez_compressed(f"{mf_run_path}/chd_elev.npz", **chd_elev_dict)

#### Save qext data to compressed file

In [68]:
np.savez_compressed(f"{mf_run_path}/qext.npz", **qext_dict)

#### Save SWMM flux data to compressed file

In [69]:
np.savez_compressed(f"{mf_run_path}/swmm_q.npz", **swmm_q_dict)